In [1]:
import numpy as np
from tqdm import tqdm
from tqdm import trange
import jsonlines
from scipy import sparse
from os import listdir
import itertools
import re
from collections import Counter
import json
from scipy.sparse import load_npz

from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

In [2]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())

### Тут tf-idf

In [3]:
ids = sorted([item['id'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')])
X = load_npz("sources/text_tfidf.npz")

In [4]:
kf = KFold(n_splits=3, shuffle=True, random_state=27)


train_index, test_index = 0, 0
for item in kf.split(X):
    train_index, test_index = item[0], item[1]
    break
    
    
print("TRAIN:", train_index, "TEST:", test_index)
print(len(train_index), len(test_index))

TRAIN: [    0     1     2 ... 96790 96792 96793] TEST: [    3     4     7 ... 96785 96789 96791]
64529 32265


In [8]:
y_predict = np.zeros((len(ok), len(test_index)), dtype=int)
clfs = []
ids_train = [ids[i] for i in train_index]
X_train = X[train_index]
X_test = X[test_index]

for i, category in enumerate(tqdm(ok)):
    clf_logreg = LogisticRegression()
    y_train = np.array([1 if category in categories_dict[id] else 0 for id in ids_train])
    if sum(y_train) > 0:
        clf_logreg.fit(X_train, y_train)
        res = clf_logreg.predict(X_test)
        clfs.append(clf_logreg)
        y_predict[i] = res
    else:
        clfs.append(None)

  0%|                                                                              | 5/14697 [00:04<3:58:48,  1.03it/s]


KeyboardInterrupt: 